## 수정 사항
1. 백업 스토어 설정에서 InMemoryDocstore를 사용해보기
2. 파라미터 조정
3. 프롬프트에 명령 추가



## 1. 백터 스토어 설정에서 InMemoryDocstore를 사용해보기

## 2. 파라미터 조정

- 모델 지정에서 파라미터 추가(temperature, max_tokens)
- 리트리버 변환에서 k값 변경(1->2)
- 모델 로드에 UTF-8 추가


### 리트리버 파라미터 설명
- search_type
 - similarity: 벡터간 유사도, 내적 사용 검색
 - mmr: 다양성 고려 검색
 - approx: 근사적 검색(대규모 데이터셋에서 빠른 검색)

- search_kwargs
 - k: 반환할 결과의 수 
 - lambda_mult: 유사도와 다양성 간의 균형 조정

# 수정 본

In [1]:
# KEY설정
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

# 모델로드
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model = "gpt-4o-mini", temperature = 0.3, max_tokens = 1000)

# 파일 로드
from langchain.document_loaders import PyPDFLoader

pdf_path = "초거대언어모델연구동향.pdf"

loader = PyPDFLoader(pdf_path)

docs = loader.load()

utf8_docs = [doc.page_content.encode('utf-8').decode('utf-8') for doc in docs]

# 문서 청크 나누기 2
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

    
# 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")

# 벡터 스토어 생성
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss

vector_dim = len(embeddings.embed_query("example text")) 
index = faiss.IndexFlatL2(vector_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

for split in splits:
    vector_store.add_texts([split.page_content], metadatas=[split.metadata])

# 리트리버 변환

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# 템플릿 정의
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an artificial intelligence model researcher. The document describes the latest research trends in super-large language models, including advancements, challenges, and key contributions. Your task is to analyze the document and provide a detailed and insightful response."),
    ("user", "Context: {context}\n\nQuestion: {question}\n\nAnswer: Please provide a comprehensive response, focusing on the key trends and research findings mentioned in the document.")
])

# 질문 응답 체인 구성
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# 질문 반복 처리
rag_chain_debug = {
    "context": retriever,   # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()     # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()| contextual_prompt | model

# 출력
    print('=====================')
    query = input("질문을 입력하세요: ")

    response = rag_chain_debug.invoke(query)
    
    print("Final Response: ")
    print(response.content)

OpenAI API key 입력: ········
질문을 입력하세요: 최근 초대형 언어모델 주요 연구동향을 알려줘
Debug Output: 최근 초대형 언어모델 주요 연구동향을 알려줘
Debug Output: {'context': [], 'question': '최근 초대형 언어모델 주요 연구동향을 알려줘'}
Final Response: 
최근 초대형 언어모델(LLM) 연구동향은 여러 가지 중요한 발전과 도전 과제를 포함하고 있습니다. 다음은 주요 연구 동향과 발견 사항에 대한 종합적인 분석입니다.

1. **모델 크기와 성능**: 초대형 언어모델의 크기가 커짐에 따라 성능이 향상되는 경향이 있습니다. 연구자들은 파라미터 수가 많은 모델이 더 복잡한 언어 패턴을 학습하고 더 정교한 응답을 생성할 수 있다는 것을 발견했습니다. 그러나 모델의 크기가 증가함에 따라 훈련 비용과 에너지 소비도 비례적으로 증가하는 문제도 제기되고 있습니다.

2. **효율적인 훈련 기법**: 최근 연구에서는 훈련 효율성을 높이기 위한 다양한 기법들이 개발되고 있습니다. 예를 들어, 혼합 정밀도 훈련(mixed-precision training)과 같은 기술이 도입되어 메모리 사용량을 줄이고 훈련 속도를 높이는 데 기여하고 있습니다. 또한, 분산 훈련 기법이 발전하여 대규모 데이터셋을 효과적으로 처리할 수 있는 방법이 모색되고 있습니다.

3. **전이 학습과 파인튜닝**: 초대형 언어모델은 전이 학습을 통해 다양한 태스크에 적응할 수 있는 능력을 보여주고 있습니다. 연구자들은 사전 훈련된 모델을 특정 도메인이나 태스크에 맞게 파인튜닝하는 방법을 통해 성능을 극대화하고 있습니다. 이 과정에서 적은 양의 데이터로도 높은 성능을 달성할 수 있는 가능성이 열리고 있습니다.

4. **윤리적 고려사항**: 초대형 언어모델의 사용이 증가함에 따라 윤리적 문제도 중요한 연구 주제로 떠오르고 있습니다. 편향(bias), 개인정보 보호, 그리고 모델의 오용 가능성에 대한 우려가 커지고 있으며, 이를 해

KeyboardInterrupt: Interrupted by user

# 1.
- 질문: 최근 초거대 언어모델 연구동향을 알려줘
- 대답: 깔끔하게 정리 되었고 내용에 맞는 대답을 한다.

# 2. 
- 질문: 각 주제들 중에 가장 중요하다고 생각드는건 무엇이며 왜그런지 설명해주세요
- 대답: 갑자기 영어로 대답을하여 놀랐다. 내용은 초대형 언어 모델 배포의 윤리적 영향과 사회적 영향을 말하고있다.

# 3.
- 질문: 방금전 글을 한글로 말해줘
- 대답: 첫 번째 질문을 그대로 가져왔다. 

# 4.
- 질문: 각 주제들 중에 가장 중요하다고 생각드는건 무엇이며 왜그런지 한글로 설명해주세요 (한글로 설명 부탁)
- 대답: 질문에 관련없는 내용이 나옴


## 프롬프트에서 바꾸어야할 점

### 이전 프롬프트
- system: 인공지능 모델 연구자이십니다. 이 문서에서는 발전, 과제 및 주요 기여를 포함하여 초대형 언어 모델의 최신 연구 동향을 설명합니다. 귀하의 임무는 문서를 분석하고 상세하고 통찰력 있는 답변을 제공하는 것입니다.
- user: 본 문서에 언급된 주요 동향과 연구 결과를 중심으로 포괄적인 답변을 부탁드립니다.